# Module 3 Assignment


#### Pricing Asian Options options using Monte Carlo

Use the expected value of the discounted payoff under the risk‐neutral density
$\mathbb{Q}$

$$ V (S; t) = E^\mathbb{Q} \left [e^{‐\int_{t}^{T} r_{\tau} d\tau} \mathbf{Payoff}
(S_T) \right] $$
for the appropriate form of payoff.

Consider payoffs of the form $ \{Put, Call\} \times \{Fixed Strike, Floating
Strike\} \times \{Arithmetic, Geometric\} $

Today's stock price $S_0 = 100$

Strike $E = 100$

Time to expiry $(T ‐ t) = 1\, year$

volatility $\sigma = 20\%$

constant risk‐free interest rate $r = 5\%$

In [1]:
%AddJar -magic http://www.brunelvis.org/jar/spark-kernel-brunel-all-1.1.jar

Starting download from http://www.brunelvis.org/jar/spark-kernel-brunel-all-1.1.jar
Finished download of spark-kernel-brunel-all-1.1.jar


In [2]:
%AddDeps --transitive org.spire-math spire_2.10 0.11.0

Marking org.spire-math:spire_2.10:0.11.0 for download
Preparing to fetch from:
-> file:/tmp/.ivy2/
-> https://repo1.maven.org/maven2
-> New file at /tmp/.ivy2/https/repo1.maven.org/maven2/org/typelevel/machinist_2.10/0.4.1/machinist_2.10-0.4.1.jar
-> New file at /tmp/.ivy2/https/repo1.maven.org/maven2/org/spire-math/spire-macros_2.10/0.11.0/spire-macros_2.10-0.11.0.jar
-> New file at /tmp/.ivy2/https/repo1.maven.org/maven2/org/spire-math/spire_2.10/0.11.0/spire_2.10-0.11.0.jar


In [3]:
%AddDeps --transitive org.scalanlp breeze_2.10 0.12.0

Marking org.scalanlp:breeze_2.10:0.12.0 for download
Preparing to fetch from:
-> file:/tmp/.ivy2/
-> https://repo1.maven.org/maven2
-> Failed to resolve org.scalanlp:breeze_2.10:0.12.0
    -> Not found: /tmp/.ivy2/cache/org.scalanlp/breeze_2.10/ivy-0.12.0.xml
    -> Not found: https://repo1.maven.org/maven2/org/scalanlp/breeze_2.10/0.12.0/breeze_2.10-0.12.0.pom


Given $z\sim\mathcal{N}(0,1)$, and using Milstein, the paths are generated iteratively by

$$ S(t_{n+1}) = S(t_{n}) \cdot \left \{1 + \sigma z \sqrt{\Delta t} + \left(r ‐ d +
\frac{1}{2}\sigma^2 \left [z^2 ‐ 1\right ]\right) \Delta t \right \} $$

A lag-4 [Complementary Multiply-With-Carry](https://en.wikipedia.org/wiki/Multiply-with-carry#Complementary-multiply-with-carry_generators) \[Marsaglia & Zaman, 1991\] is used to generate uniform psuedo-random numbers, which are then transformed using the [Ziggurat Algorithm](https://en.wikipedia.org/wiki/Ziggurat_algorithm) \[Marsaglia & Tsang, 2000\] to sample $z$.

In [4]:
import spire.implicits._
import sqlContext.implicits._

In [5]:
def milsteinPaths(s0:Double, sigma:Double, r:Double, d: Double, steps:Int, t:Double, antithetic:Boolean = true) = {
  import spire.random._  
    
  val half_sigma_2 = sigma*sigma/2.0
  val drift = r - d
  val cmwc5 = rng.Cmwc5.fromTime()
  val dt = t / steps
    
  def step(s:Double, z:Double) : Double =
    s * {1 + sigma*z*math.sqrt(dt) + (drift + half_sigma_2*(z*z - 1))*dt}
    
  val zs = Stream.continually(Ziggurat.rnor(cmwc5)).take(steps).toArray
  val a1 = zs.scanLeft(s0)(step)
  lazy val a2 = zs.map(_ * -1.0).scanLeft(s0)(step)
  a1 #:: (if(antithetic) Stream(a2) else Stream.empty)
}

In [6]:
sealed trait Verb
case object Put extends Verb
case object Call extends Verb

In [7]:
sealed trait TradeType
case class FloatingRate(strike:Double) extends TradeType
case object FloatingStrike extends TradeType

In [8]:
type Averaging = Array[Double] => Double
val arithmetic : Averaging =  d => d.sum / d.size
val geometric : Averaging = d => math.exp(d.map(math.log).qsum / d.size)
val lastVal : Averaging = d => d.last

In [9]:
def payoff(verb: Verb, tType: TradeType, averaging: Averaging, data: Array[Double]) : Double = {
  val average = averaging(data)
  (verb, tType) match {
    case (Put, FloatingRate(strike)) => math.max(strike - average, 0.0)
    case (Call, FloatingRate(strike)) => math.max(average - strike, 0.0)
    case (Put, FloatingStrike) => math.max(average - data.last, 0.0)
    case (Call, FloatingStrike) => math.max(data.last - average, 0.0)
  } 
}

In [10]:
def getPaths(s0: Double, sigma: Double, r:Double, d:Double, n: Int, steps: Int, T: Double, useAntithetic: Boolean) = {
  Stream.continually(milsteinPaths(s0, sigma, r, d, steps, T, useAntithetic))
  .flatten
  .take(n)
}

In [11]:
def simulate(
     v: Verb, s: TradeType, a: Averaging, n: Int, r: Double, d: Double,
     s0: Double, sigma: Double, steps: Int, T: Double, 
     useAntithetic : Boolean = true
) = {
  val sum = 
    getPaths(s0, sigma, r, d, n, steps, T, useAntithetic)
    .map(payoff(v, s, a, _))
    .qsum

  math.exp(-r*T)*(sum / n)
}

In [12]:
def makeLine = milsteinPaths(100.0, 0.2, 0.05, 0.0, 365, 1.0, antithetic = false).head

In [13]:
case class PPoint(series: String, x:Double, y:Double)

In [14]:
val d4 = (1 to 100) flatMap (idx => makeLine.zipWithIndex.map{case (d, i) => PPoint(idx.toString, i, d)})
val ts3 = d4.toDF

In [15]:
%%brunel data('ts3') x(x) y(y) bin(x:75, y:75) color(#count) interaction(none)

Fig 1. 100 sample milstein GBM paths.


Implementing the antithetic variates technique is then a relatively straightforward enhancement:

For every draw $z\sim\mathcal{N}(0,1)$, we instead return $\{z, -z\}$ and generate two, complementary, paths. This means we generate paired paths, one along $\{z_1, z_2, z_3, ... z_n\}$, the other along $\{-z_1, -z_2, -z_3, ... -z_n\}$.

Note that since the $\frac{1}{2}\sigma^2 \left [z^2 ‐ 1\right ]$ term that the Milstein sceme adds to the Euler one, will result in some amount of positive covariance between the two paths. Given this, we would expect that the Euler scheme would benefit more from antithetic variates technique than the Milstein scheme would.

In [16]:
def antithetics = Stream.continually(milsteinPaths(100.0, 0.2, 0.05, 0.0, 365, 1.0, antithetic = true)).flatten

In [17]:
val d5 = antithetics.
        take(100).
        zipWithIndex.
        flatMap{case (ds, idx) => ds.zipWithIndex.map{case (d,i) => PPoint(idx.toString,i,d)}}
val ts5 = sqlContext.createDataFrame(d5)

In [18]:
%%brunel data('ts5') x(x) y(y) bin(x:75, y:75) color(#count) interaction(none)

Fig 2. 100 sample paths using Antithetic Variates and Milstein

### Vanilla Options

One can view vanilla european options as being analogous to asian options with a
single, discrete averaging observation at $t=T$.

Since our averaging function is of the form $f:\mathbb{R}^{n}\mapsto \mathbb{R}$, by discarding the first $n‐1$ samples and returning the final sample, we should price vanilla options correctly.

For an at‐the‐money call, with $S_0 = 100$, $\sigma = 20\%$, $r = 5\%$ and 1 year to
maturity, Black‐Scholes gives $V= 10.4506$

In [19]:
def simulatePath(
     v: Verb, s: TradeType, a: Averaging, n: Int, r: Double, d: Double,
     s0: Double, sigma: Double, steps: Int, T: Double, 
     useAntithetic : Boolean = true
) = {
  val csums = 
    getPaths(s0, sigma, r, d, n, steps, T, useAntithetic)
    .map(payoff(v, s, a, _))
    .scanLeft(0.0)(_+_)
  val df = math.exp(-r*T)
  csums.zipWithIndex.map{ case (s, i) => df*(s / i)}.drop(1)
}

In [20]:
case class MCPathPoint(analytic:Double, mc:Double, iters:Double)

In [21]:
def getSimulationFrame(analytic:Double, v: Verb, s: TradeType, a: Averaging, antithetic : Boolean = true) = {
  simulatePath(v, s, a, 2000, 0.05, 0.0, 100.0, 0.2, 365, 1.0, useAntithetic = antithetic)
  .zipWithIndex.map{ case (d, i) => MCPathPoint(analytic, d, i + 1)}
  .toDF
}

In [22]:
val atmVanillaCall = getSimulationFrame(10.4506, Call, FloatingRate(strike=100), lastVal)

In [23]:
%%brunel data('atmVanillaCall') 
x(iters:log) y(analytic, mc) color(#series) axes(y, x:'# of iterations') line interaction(none)

the corresponding Put option is $V = 5.5735$ under Black‐Scholes

In [24]:
val atmVanillaPut = getSimulationFrame(5.5375, Put, FloatingRate(strike=100), lastVal)

In [25]:
%%brunel data('atmVanillaPut')
x(iters:log) y(analytic, mc) color(#series) axes(y, x:'# of iterations') line interaction(none)

These results are in close agreement with the analytical solution, suggesting that
the path generation is working well.

### Geometric Averaging

In the continuously sampled Geometric Payoff case, there is a closed form analytic
expression for the price [Kemna et Vorst, 1990]. This provides a reference benchmark for the Geometric case.

$$ C =
S_0 e^{‐\left(r + \frac{\sigma^2}{6}\right)\frac{T}{2}}N
\left( \frac{ \ln \frac{S_0}{K} + \left(r + \frac{\sigma^2}{6} \right) \frac{T}{2}
}{\sigma\sqrt{T/3}} \right)
‐ Ke^{‐rT}N\left( \frac{ \ln \frac{S_0}{K} + \left(r ‐ \frac{\sigma^2}{2} \right)
\frac{T}{2}  }{\sigma\sqrt{T/3}} \right)
$$

In [26]:
def kemnaVorst(k:Double, s0:Double, sigma:Double, T: Double, r:Double) = {
  import math._
  val N = breeze.stats.distributions.Gaussian.distribution((0.0, 1.0))
  val vari = sigma*sigma 
  val c1 = s0*exp(-(r+vari/6.0)*T/2.0)
  val c2 = k*exp(-r*T)
  val a1 = (log(s0/k) + (r + vari/6.0)*T/2.0) / (sigma*sqrt(T/3.0))
  val a2 = (log(s0/k) + (r - vari/2.0)*T/2.0) / (sigma*sqrt(T/3.0))
  
  (c1* N.cdf(a1) - c2* N.cdf(a2), c2 * N.cdf(-a2) - c1 * N.cdf(-a1))
}

In [27]:
val (callPrice, putPrice) = kemnaVorst(100.0, 100.0, 0.20, 1.0, 0.05)

In [28]:
val atmGeoCall = getSimulationFrame(callPrice, Call, FloatingRate(strike=100), geometric)

In [29]:
%%brunel data('atmGeoCall')
x(iters:log) y(analytic, mc) color(#series) axes(y, x:'# of iterations') line interaction(none)

In [30]:
val atmGeoPut = getSimulationFrame(putPrice, Put, FloatingRate(strike=100), geometric)

In [31]:
%%brunel data('atmGeoPut')
x(iters:log) y(analytic, mc) color(#series) axes(y, x:'# of iterations') line interaction(none)

For arithmetic averaging, we have no analytic solution, but approximations are known:

In [32]:
val atmArithmCall = getSimulationFrame(0.0, Call, FloatingRate(strike=100), arithmetic)

In [33]:
%%brunel data('atmArithmCall')
x(iters:log) y(mc) axes(y:'Price', x:'# of iterations') line interaction(none)

In [34]:
val atmArithmPut = getSimulationFrame(0.0, Put, FloatingRate(strike=100), arithmetic)

In [35]:
%%brunel data('atmArithmPut')
x(iters:log) y(mc) axes(y:'Price', x:'# of iterations') line interaction(none)

### Antithetic Variates

In [36]:
val atmGeoCallNoAntithetic = getSimulationFrame(5.5468, Call, FloatingRate(strike=100), geometric, antithetic=false)

In [37]:
%%brunel 
data('atmGeoCallNoAntithetic')
x(iters:log) y(mc) axes(y:'Price', x:'# of iterations') line interaction(none) +
data('atmGeoCall')
x(iters:log) y(mc) axes(y:'Price', x:'# of iterations') line interaction(none)

# TODO

* floating strikes
* add stats summary per run
* Use student T to get confidence intervals for the means
* Sobol?


### Floating Strike Options

In [38]:
val atmGeoCallFloatingStrike = getSimulationFrame(0.0, Call, FloatingStrike, geometric)

In [39]:
%%brunel data('atmGeoCallFloatingStrike')
x(iters:log) y(mc) axes(y:'Price', x:'# of iterations') line interaction(none)

In [40]:
val atmArithCallFloatingStrike = getSimulationFrame(0.0, Call, FloatingStrike, arithmetic)

In [41]:
%%brunel data('atmArithCallFloatingStrike')
x(iters:log) y(mc) axes(y:'Price', x:'# of iterations') line interaction(none)

In [42]:
val atmGeoPutFloatingStrike = getSimulationFrame(0.0, Put, FloatingStrike, geometric)

In [43]:
%%brunel data('atmGeoPutFloatingStrike')
x(iters:log) y(mc) axes(y:'Price', x:'# of iterations') line interaction(none)

In [44]:
val atmArithPutFloatingStrike = getSimulationFrame(0.0, Put, FloatingStrike, arithmetic)

In [45]:
%%brunel data('atmArithPutFloatingStrike')
x(iters:log) y(mc) axes(y:'Price', x:'# of iterations') line interaction(none)

## Pricing Summary

In [49]:
def finalPrice(df:org.apache.spark.sql.DataFrame) = df.orderBy($"iters".desc).head.getDouble(1)
println(finalPrice(atmGeoPut))
println(finalPrice(atmGeoCall))
println(finalPrice(atmArithmPut))
println(finalPrice(atmArithmCall))
println(finalPrice(atmGeoPutFloatingStrike))
println(finalPrice(atmGeoCallFloatingStrike))
println(finalPrice(atmArithPutFloatingStrike))
println(finalPrice(atmArithCallFloatingStrike))

3.402360601703008
5.519910907773018
3.415631217567175
5.955218579062078
3.2445166679825146
5.953693606226353
3.393644006529303
5.707143732393586


For the conditions specified in the introduction:

|Verb|Averaging |FloatType|Price|
|----|----------|---------|-----|
|Put |Geometric |Rate     |3.4023     |
|Call|Geometric |Rate     |5.5199     |
|Put |Arithmetic|Rate     |3.4156     |
|Call|Arithmetic|Rate     |5.9552     |
|Put |Geometric |Strike   | 3.2445    |
|Call|Geometric |Strike   |5.9537     |
|Put |Arithmetic|Strike   |3.3936     |
|Call|Arithmetic|Strike   |5.7071  |

